# Minsait Land Classification

Modelo de clasificación automática de suelos en base a imágenes de satélite.
Grupo **Astralaria** del centro **Universitat Poltècnica de València** formado por **Asier Serrano Aramburu** y **Mario Campos Mocholí**

## 1. Procesamiento de los datos

#### 1.1 Consideraciones previas
Podemos observar claramente que tenemos un conjunto de muestras desbalanceado y debemos ajustarlo para que el modelo a entrenar no se especialice solo en la clase *RESIDENTIAL* e ignore las demás. En nuestro caso hemos optado por un equilibrado al por menor explicado más adelante.

#### 1.2 Preprocesamiento
Para entrenar nuestro modelo todas las variables deben ser numéricas, pero tras observar los datos descubrimos que una reducida cantidad de muestras que contienen variables alfanuméricas o simplemente vacías. Podríamos haber optado por eliminar estas muestras por considerarlas corruptas, pero tras un análisis posterior descubrimos que son muy útiles, sobre todo para la clase *AGRICULTURE* por lo que optamos por completarlas y adaptarlas.

In [2]:
import numpy as np

# Diccionario para codificar los nombres de las clases
categorical_encoder_class = {'RESIDENTIAL': 0,
    'INDUSTRIAL': 1,
    'PUBLIC': 2,
    'OFFICE': 3,
    'OTHER': 4,
    'RETAIL': 5,
    'AGRICULTURE': 6
}

# Diccionario para codificar las variables no numéricas
categorical_encoder_catastral = {'A': -10,
    'B': -20,
    'C': -30,
    '""': 50
}

# Variable que contendrá las muestras
data = []

with open(r'Data\Modelar_UH2020.txt') as read_file:
    # La primera linea del documento es el nombre de las variables, no nos interesa
    read_file.readline()
    # Leemos línea por línea adaptando las muestras al formato deseado (codificar el valor catastral y la clase)
    for line in read_file.readlines():
        # Eliminamos el salto de línea final
        line = line.replace('\n', '')
        # Separamos por el elemento delimitador
        line = line.split('|')
        if line[54] in categorical_encoder_catastral:
            line[54] = categorical_encoder_catastral[line[54]]
            if line[54] is 50:
                line[53] = -1
        line[55] = categorical_encoder_class[line[55]]
        # No nos interesa el identificador de la muestra, lo descartamos
        data.append(line[1:])

# Finalmente convertimos las muestras preprocesadas a una matriz
data = np.array(data).astype('float32')

#### 1.3 Equilibrado
Una vez preprocesados los datos en un formato deseable para entrenar modelos, procedemos a un equilibrado de estos al por menor. Contamos con 102892‬ muestras repartidas en 90173 de la clase *RESIDENTIAL*, 4490 de *INDUSTRIAL*, 2976 de tipo *PUBLIC*, 1828 de la clase *OFFICE*, 1332 de tipo *OTHER*, 2093 de *RETAIL* y 338 de la clase *AGRICULTURE*. Después de muchas pruebas hemos optado por añadir todas las muestras de cada clase excepto la de *RESIDENTIAL* que solo añadiremos **X**. Añadir más produce especialización hacía esa clase y empeora la precisión individual de las otras.

In [4]:
# Variable que contendrá las muestras separadas por clase
data_per_class = []

# Añadimos una lista vacía por clase
for _ in range(7):         
    data_per_class.append([])
# Añadimos a la lista de cada clase las muestras de esta
for sample in data:
    data_per_class[int(sample[54])].append(sample)

# Variable que contendrá los datos procesados
data_proc = []

# Muestras de la clase RESIDENTIAL
data_proc += data_per_class[0][0:12000]
# Muestras de las otras clases
for i in range(6):
    data_proc += data_per_class[i + 1]
    
# Volvemos a convertir los datos una vez procesados a una matriz y los mezclamos
data_proc = np.array(data_proc)
np.random.shuffle(data_proc)

#### 1.4 División
Finalmente separamos nuestro conjunto de muestras en dos conjuntos, uno de entrenamiento y otro de validación. El tamaño de este último vendrá dado por la variable *test_avg*.

In [5]:
from sklearn.model_selection import train_test_split

# Variable en el rango (0.0 - 1.0) que indica el procentaje de muestras de validación
test_avg = 0.2

X_train, X_test, y_train, y_test = train_test_split(data_proc[:, :54], data_proc[:, 54], test_size = test_avg)

## 2. Entrenamiento del modelo

#### 2.1 Consideraciones previas
Contamos con un conjunto de entrenamiento considerable donde cada muestra tiene también un número elevado de variables y ademas siete clases a predecir. Es inutil entrenar un modelo basado en un separador lineal o alguna tecnica de clustering (ya que tenemos las muestras etiquetadas). Hemos optado por realizar un modelo basado en dos submodelos y por iteraciones, es decir, cada modelo sera entrenado varias veces y obtendremos su predicción local, seleccionando como predicción final aquella que se haya predicho más veces. **NOTA: Cada apartado se debera ejecutar secuencialmente pero solo a partir del 2.6 se obtendran resultados validos

#### 2.2 Entrenamiento modelo XGB
https://xgboost.readthedocs.io/en/latest/index.html

In [6]:
import xgboost as xgb

model = xgb.XGBClassifier(max_depth=None, learning_rate=0.1, n_estimators=400, verbosity=None, objective=None, 
        booster=None, tree_method=None, n_jobs=-1, gamma=None, min_child_weight=None, max_delta_step=None, 
        subsample=None, colsample_bytree=None, colsample_bylevel=None, colsample_bynode=None, reg_alpha=None, 
        reg_lambda=None, scale_pos_weight=None, base_score=None, random_state=None)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#### 2.3 Entrenamiento modelo RandomForest
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [7]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=400, criterion='entropy', max_depth=60, min_samples_split=5, 
        min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='log2', max_leaf_nodes=None, 
        min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=-1, 
        random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#### 2.4 Evaluación
En la Librería Sklearn encontramos diversas funciones útiles para evaluar nuestros sistemas.

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Util para observar clases con poca separación entre si
print('Matriz de confusión:\n{}\n\n'.format(confusion_matrix(y_test, y_pred)))
# Útil para conocer la precisión de cada clase y demás estadisticas
print('Informe de clasificación:\n{}\n'.format(classification_report(y_test, y_pred)))
# Útil para conocer la precisión global del modelo
print('Precisión: {}'.format(accuracy_score(y_test, y_pred)))

Matriz de confusión:
[[2200   22   72   22    9   15    2]
 [ 163  646   41   31    9    9    8]
 [ 185   42  309   30   42   14    3]
 [ 127   81   52  113    9    9    1]
 [  71   21   46    5  131    2    0]
 [ 199   42   35   17    8   99    1]
 [   6   12    3    3    0    1   44]]


Informe de clasificación:
              precision    recall  f1-score   support

         0.0       0.75      0.94      0.83      2342
         1.0       0.75      0.71      0.73       907
         2.0       0.55      0.49      0.52       625
         3.0       0.51      0.29      0.37       392
         4.0       0.63      0.47      0.54       276
         5.0       0.66      0.25      0.36       401
         6.0       0.75      0.64      0.69        69

    accuracy                           0.71      5012
   macro avg       0.66      0.54      0.58      5012
weighted avg       0.69      0.71      0.68      5012


Precisión: 0.7067039106145251


#### 2.5 Predicción 
Una vez entrenado, cada modelo predecirá cada variable a estimar y almacenara la predicción para su evaluación posterior.

In [9]:
# Variable que contendrá las muestras a predecir
data_predict = []

# Mismo procesamiento de datos que para el conjunto inicial
with open(r'Data\Estimar_UH2020.txt') as read_file:
    # La primera línea del documento es el nombre de las variables, no nos interesa
    read_file.readline()
    # Leemos línea por línea adaptando las muestras al formato deseado (codificar el valor catastral)
    for line in read_file.readlines():
        line = line.replace('\n', '')
        line = line.split('|')
        if line[54] in categorical_encoder_catastral:
            line[54] = categorical_encoder_catastral[line[54]]
            if line[54] is 50:
                line[53] = -1
        data_predict.append(line)

# Finalmente convertimos las muestras preprocesadas a una matriz (no numérica, nos interesa el id esta vez)
data_predict = np.array(data_predict)

# Lista auxiliar que contendrá las predicciones locales de cada modelo
predictions_aux = model.predict(data_predict[:, 1:].astype('float32'))

# Variable que contendrá las predicciones globales de cada muestra
predictions = {}

# Añadimos a las predicciones globales la predicción del modelo local
for i in range(len(data_predict)):
    if (data_predict[i, 0] not in predictions):
        predictions[data_predict[i, 0]] = [int(predictions_aux[i])]
    else:
        predictions[data_predict[i, 0]].append(int(predictions_aux[i]))

#### 2.6 Modelo global
Finalmente entrenamos nuestro multi-modelo tantas veces como la variable *iterations* indique. Este apartado es la fusión de los anteriores para conseguir replicar el sistema de entrenamiento. La variable *debug_mode* indica si en cada entrenamiento, el modelo mostrara estadísticas o no.

In [42]:
# Número de iteraciones total por modelo
iterations = 20

# Variable anterior, inicializada de nuevo
predictions = {}

# Si True, muestra información de cada modelo local tras entrenarlo
debug_mode = False

for ite in range(iterations):
    # Mostramos el porcentaje de entrenamiento
    print('Entrenamiento completo al {}%'.format(ite/iterations * 100))
    
    np.random.shuffle(data_proc)
    X_train, X_test, y_train, y_test = train_test_split(data_proc[:, :54], data_proc[:, 54], test_size = test_avg)
    
    # Modelo XGB
    model = xgb.XGBClassifier(max_depth=None, learning_rate=0.1, n_estimators=400, verbosity=None, objective=None, 
        booster=None, tree_method=None, n_jobs=-1, gamma=None, min_child_weight=None, max_delta_step=None, 
        subsample=None, colsample_bytree=None, colsample_bylevel=None, colsample_bynode=None, reg_alpha=None, 
        reg_lambda=None, scale_pos_weight=None, base_score=None, random_state=None)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    if debug_mode:
        print('Matriz de confusión:\n{}\n'.format(confusion_matrix(y_test, y_pred)))
        print('Informe de clasificación:\n{}\n'.format(classification_report(y_test, y_pred)))
        
    predictions_aux = model.predict(data_predict[:, 1:].astype('float32'))
    for i in range(len(data_predict)):
        if (data_predict[i, 0] not in predictions):
            predictions[data_predict[i, 0]] = [int(predictions_aux[i])]
        else:
            predictions[data_predict[i, 0]].append(int(predictions_aux[i]))
        
    # Modelo RandomForest
    model = RandomForestClassifier(n_estimators=400, criterion='entropy', max_depth=60, min_samples_split=5, 
        min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='log2', max_leaf_nodes=None, 
        min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=-1, 
        random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    if debug_mode:
        print('Matriz de confusión:\n{}\n'.format(confusion_matrix(y_test, y_pred)))
        print('Informe de clasificación:\n{}\n'.format(classification_report(y_test, y_pred)))
        
    predictions_aux = model.predict(data_predict[:, 1:].astype('float32'))
    for i in range(len(data_predict)):
        if (data_predict[i, 0] not in predictions):
            predictions[data_predict[i, 0]] = [int(predictions_aux[i])]
        else:
            predictions[data_predict[i, 0]].append(int(predictions_aux[i]))
print('Entrenamiento completo')

Entrenamiento completo al 0.0%
Entrenamiento completo al 5.0%
Entrenamiento completo al 10.0%
Entrenamiento completo al 15.0%
Entrenamiento completo al 20.0%
Entrenamiento completo al 25.0%
Entrenamiento completo al 30.0%
Entrenamiento completo al 35.0%
Entrenamiento completo al 40.0%
Entrenamiento completo al 45.0%
Entrenamiento completo al 50.0%
Entrenamiento completo al 55.00000000000001%
Entrenamiento completo al 60.0%
Entrenamiento completo al 65.0%
Entrenamiento completo al 70.0%
Entrenamiento completo al 75.0%
Entrenamiento completo al 80.0%
Entrenamiento completo al 85.0%
Entrenamiento completo al 90.0%
Entrenamiento completo al 95.0%
Entrenamiento completo


## 3.0 Evaluación global
Finalmente obtenemos la predicción final de cada muestra a predecir.

In [10]:
# Diccionario para decodificar el nombre de las clases
categorical_decoder_class = {0: 'RESIDENTIAL',
    1: 'INDUSTRIAL',
    2: 'PUBLIC',
    3: 'OFFICE',
    4: 'OTHER',
    5: 'RETAIL',
    6: 'AGRICULTURE'}

# Función que obtiene la moda de una lista
def most_frequent(lst): 
    return max(set(lst), key = lst.count) 

with open(r'Minsait_Universitat Politècnica de València_Astralaria.txt', 'w') as write_file:
    # Nombre de las variables respuesta
    write_file.write('ID|CLASE\n')
    # Para cada muestra obtenemos su moda y volcamos el resultado a un fichero respuesta
    for sample in data_predict:
        write_file.write('{}|{}\n'.format(sample[0], categorical_decoder_class[most_frequent(predictions[sample[0]])]))

## 4.0 Guardado del modelo